## Code

In [1]:
%load_ext ipython_sparql_pandas

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
from random import randint
sparql = SPARQLWrapper('http://MacBook-Air-van-Daphne.local:7200/repositories/statements')
sparql.setReturnFormat(JSON)

In [3]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .',
        '?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Country> .']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>',
              '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/Country>'
              ]

def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred = []
    listOfCompQ = []
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])
        listOfPred.append(attr['p'])
        listOfCompQ.append(attr['isCompound'])
    return(listOfObj,listOfPred,listOfCompQ)

def generateQuestion(listOfPred, listOfObj, listOfCompQ, index, subject):
    flag = True
    if ('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>') in AttrHistory:
        return generateQuestion(listOfPred, listOfObj, listOfCompQ, index+1, subject)
        
    else:
        AttrHistory.append('<' + str(listOfPred[index]['value']) +'> <'+str(listOfObj[index]['value'])+ '>')
        
    try:
        int(listOfObj[index]['value'][0])
    except ValueError:
           flag = False
    
    if listOfCompQ[index]['value'] == 'false':    
        print(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} {listOfObj[index]["value"].split("/")[-1]}?' )
    
    elif listOfCompQ[index]['value'] == 'true' :   
        print(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} of type {listOfObj[index]["value"].split("/")[-1]}?' )
    
    if flag is True:
        datatype = listOfObj[index]["datatype"].split("#")[-1]
        answer = findAnswer(subject, '<' + str(listOfPred[index]["value"]) +'>', '"'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype, listOfCompQ[index]['value'])
    else:
        answer = findAnswer(subject, '<' + str(listOfPred[index]["value"]) +'>', '<'+str(listOfObj[index]["value"])+ '>', listOfCompQ[index]['value'])
    print(answer)
    
    if answer:   
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            
            if listOfCompQ[index]['value'] == 'false':
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) + '> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'.') 
            elif listOfCompQ[index]['value'] == 'true':
                # NOG AANPASSEN
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) + '> "' + str(listOfObj[index]["value"]) + '"^^xsd:'+datatype+'.') 
        
        else:
            if listOfCompQ[index]['value'] == 'false':
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>.')
            elif listOfCompQ[index]['value'] == 'true':
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> [a <'+str(listOfObj[index]["value"])+ '>].')
        
        
    elif not answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            
            if listOfCompQ[index]['value'] == 'false':
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
            elif listOfCompQ[index]['value'] == 'true':
                # NOG AANPASSEN
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
        
        else:
            if listOfCompQ[index]['value'] == 'false':
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>. }}')   
            elif listOfCompQ[index]['value'] == 'true':
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> [a <'+str(listOfObj[index]["value"])+ '>]. }}')   
                
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)

  
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
        SELECT (count(*) as ?count) ?p ?o ?isCompound WHERE {{
        {PosFilters}
        {{?s ?p [a ?o] . BIND (true as ?isCompound) }} UNION {{?s ?p ?o . BIND (false as ?isCompound) }}
        {NegFilters}
        filter (?o != <http://www.w3.org/2002/07/owl#Class>)
        filter (?o != <http://example.com/popularEntity> )
        }}
        group by ?p ?o ?isCompound
        order by abs({left} - ?count )
        limit 20            
        """) 
    return FilterQuestion

In [4]:
def pickSubject():
    ind=randint(0,12500)
    query = f"""
    select distinct ?s
    where {{
        ?s ?p ?o .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    }}
    limit 12500
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"][ind:ind+1]:
        return("<"+r["s"]["value"]+">")

In [5]:
def findAnswer(subject,p,o,compq):
    query = f"""
        SELECT ?s ?p ?o
        WHERE {{
        {{?s ?p ?o1 . }}
        UNION
        {{?s ?p [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?o2] .}}
        FILTER(?s = {subject})
        FILTER(?p = {p})
        FILTER(if({compq}, ?o2, ?o1) = {o})
        }}"""
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    res = qres["results"]["bindings"]
    return (len(res))>0

In [6]:
def numberleft(PosFilters,NegFilters):
    query =  f"""
            select (count(?s) as ?count) where 
            {{
            {PosFilters}            
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))

In [7]:
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            
            {PosFilters}
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            
            }}
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres["results"]["bindings"]

In [8]:
# resets all variables
def game_reset():
    PosAttr= ['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
    NegAttr= ['']
    AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    left = numberleft("".join(PosAttr), "".join(NegAttr))
    FilterQuestion =  f"""
        SELECT (count(*) as ?count) ?p ?o ?isCompound WHERE {{
        {{?s ?p [a ?o] . BIND (true as ?isCompound) }} UNION {{?s ?p ?o . BIND (false as ?isCompound) }}
        ?s a <http://schema.org/Country> .
        ?s a <http://example.com/popularEntity> .
        filter (?o != <http://www.w3.org/2002/07/owl#Class>)
        filter (?o != <http://example.com/popularEntity> )
        }}
        group by ?p ?o ?isCompound
        order by abs( %s - ?count )
        limit 20
        """ %((left/2))
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred, listOfCompQ = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [ ]:
#without printing results
import random

wincount = 0
gamesplayed = 0
totalgames = 1

while gamesplayed < totalgames:
    # Resets all variables
    left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
    i=0
    outofguess = []
    print("\nGames played: " + str(gamesplayed))
    
    # pick subject
    #subject = pickSubject()
    subject = '<http://yago-knowledge.org/resource/Italy>'
    print("\n" + subject)

    while i < 20:

        # list of Pred and Obj for questions
        listOfObj ,listOfPred, listOfCompQ = extractCountAndAttr(qres)

        # if only 1 popular entity is left break
        if left<=1:
            break

        # ask question and add filters to query
        #print('Question: %s' %(i+1))
        PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, listOfCompQ, 0, subject)
        print(PosFilters)

        # number of popular entities left
        left = numberleft(PosFilters, NegFilters)

        # run query
        query = updateQuery(left/2,PosFilters, NegFilters)
        sparql.setQuery(query)
        qres = sparql.queryAndConvert()

        i+=1

    # Prints answer
    if i != 20:
        for attr in popentities(PosFilters, NegFilters):
            for x in [attr['s']['value']][0:10]:
                answer = (x.split("/")[-1])
        print("Your Answer is: " + answer)
        if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
            wincount += 1
            gamesplayed += 1
            print("Win!")
        else:
            gamesplayed += 1
            print("Lose...")

    # ran out of questions
    else:
        print('Final Question:')
        for j in popentities(PosFilters, NegFilters):
                outofguess.append(f'{j["s"]["value"].split("/")[-1]}')
        guess = random.choice(outofguess)
        print("Is it " + guess + "?")
        if str(subject.split("/")[-1]) == str(guess + ">"):
            wincount += 1
            gamesplayed += 1
            print('Win!')
        else:
            gamesplayed += 1
            print('Lose...')
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")

In [ ]:
# with count print
import random

# Resets all variables
left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
i=0
outofguess = []

while i < 20:
    
    # list of Pred and Obj for questions
    listOfObj ,listOfPred, listOfCompQ = extractCountAndAttr(qres)
    
    # if only 1 popular entity is left break
    if left<=1:
        break
     
    # ask question and add filters to query
    print('Question: %s' %(i+1))
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, listOfCompQ, 0)
    
    # number of popular entities left
    left = numberleft(PosFilters, NegFilters)
    
    # run query
    query = updateQuery(left/2,PosFilters, NegFilters)
    sparql.setQuery(query)
    qres = sparql.queryAndConvert()
    
    # prints splits for top Pred and Objs 
    print('remaining entities:', left)
    for r in qres["results"]["bindings"]:
        print(*[f'{x.split("/")[-1][:40]:40s}' for x in [r['count']["value"],r['p']["value"],r['o']["value"]]])
        
    i+=1

# Prints answer
if i != 20:
    for attr in popentities(PosFilters, NegFilters):
        print(*[f'Your Answer is: {x.split("/")[-1]}' for x in [attr['s']['value']]][0:10])
        
# ran out of questions
else:
    print('Final Question:')
    for j in popentities(PosFilters, NegFilters):
            outofguess.append(f'{j["s"]["value"].split("/")[-1][:40]:40s}')
    guess = random.choice(outofguess)
    print(guess)
    answer = (input('Is this correct?').startswith('y'))
    if answer is True:
        print('Win')
    else:
        print('game over')

In [ ]:
# with guess print
import random

# Resets all variables
left, qres, PosAttr, NegAttr, AttrHistory = game_reset()
i=0
outofguess = []

while i < 20:
   
    # list of Pred and Obj for questions
    listOfObj ,listOfPred, listOfCompQ = extractCountAndAttr(qres)
    
    # if only 1 popular entity is left break
    if left<=1:
        break
        
    # ask question and add filters to query
    print('Question: %s' %(i+1))
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, listOfCompQ, 0)
    
    # number of popular entities left
    left = numberleft(PosFilters, NegFilters)
    
    # run query
    query = updateQuery(left/2, PosFilters, NegFilters)
    sparql.setQuery(query)
    qres = sparql.queryAndConvert()
    
    # prints possible entities left
    print('Possible Guesses:')
    for subject in popentities(PosFilters, NegFilters):
        print(*[f'{x.split("/")[-1]}' for x in [subject['s']['value']]][0:10]) 
        
    print()
    i+=1

# Prints answer
if i != 20:
    for attr in popentities(PosFilters, NegFilters):
        print(*[f'Your Answer is: {x.split("/")[-1]}' for x in [attr['s']['value']]][0:10])
        
# ran out of questions
else:
    print('Final Question:')
    for j in popentities(PosFilters, NegFilters):
            outofguess.append(f'{j["s"]["value"].split("/")[-1][:40]:40s}')
    guess = random.choice(outofguess)
    print(guess)
    answer = (input('Is this correct?').startswith('y'))
    if answer is True:
        print('Win')
    else:
        print('game over')

In [ ]:
x = '1683-03-07'
int(x[0])

### Notes
- football teams 
    - some football teams do not have the type football team